<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# Pandas per la gestione dei dati
import pandas as pd

from collections import Counter

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [29]:
# Importiamo il dataset ottenuto tramite le operazioni di scraping
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/charting-m-points-2010s.csv")

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,1,0,0,0,0.0,0-0,1,True,1,4n,4b1b1b3n@,NaN,1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,2,0,0,0,0.0,15-0,1,True,1,5n,5f28f2b1b1b2f3f3d@,NaN,1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,3,0,0,0,0.0,30-0,1,True,1,4C,NaN,NaN,1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,4,0,0,0,0.0,40-0,1,True,1,4r2d#,NaN,NaN,1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,5,0,0,1,0.0,0-0,2,True,2,5d,c4b1b1d!@,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329842,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,119,0,1,3,5.0,AD-40,19,True,1,4d,4b3n#,NaN,1
329843,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,120,0,1,4,5.0,0-0,20,True,2,5d,6f1w#,NaN,2
329844,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,121,0,1,4,5.0,15-0,20,True,2,5s38b2f1f1f2b2b2f1f3b3b+1n@,NaN,NaN,2
329845,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,122,0,1,4,5.0,30-0,20,True,2,4f18f1f3b3s3b3s3f+1l1o3*,NaN,NaN,2


Analisi delle tipologie degli esiti finali dei punti, analisi dei casi rari

In [30]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['1st'][-2]

    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['2nd'][-2]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 110288
'@': 103774
'*': 102804
'd': 4828
'n': 4780
'w': 1397
'S': 1003
'R': 429
'x': 297
'C': 135
'!': 22
' ': 16
'g': 14
'Q': 12
'1': 10
'3': 9
'e': 6
'P': 5
'9': 4
'2': 2
'c': 2
'8': 2
'7': 2
'p': 1
'^': 1
'+': 1
'.': 1
'-': 1
'b': 1


Correzione dei casi rari e enfasi a informazioni importanti in un formato più gestibile.

La descrizione in formato semplice, una riga per ogni stringa:


- `1` = `noinfo_1`: il giocatore 1 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `2` = `noinfo_2`: il giocatore 2 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `3` = `serve_ace_1`: punto diretto con il servizio (ace) fatto dal giocatore 1.

- `4` = `serve_ace_2`: punto diretto con il servizio (ace) fatto dal giocatore 2.

- `5` = `serve_miss2_1`: il giocatore 1 vince il punto su un errore di servizio del giocatore 2.

- `6` = `serve_miss1_2`: il giocatore 2 vince il punto su un errore di servizio del giocatore 1.

- `7` = `rally_winner_1`: il giocatore 1 vince il punto con un colpo vincente durante lo scambio.

- `8` = `rally_winner_2`: il giocatore 2 vince il punto con un colpo vincente durante lo scambio.

- `9` = `rally_forced2_1`: il giocatore 1 forza un errore del giocatore 2 durante lo scambio.

- `A` = `rally_forced1_2`: il giocatore 2 forza un errore del giocatore 1 durante lo scambio.

- `B` = `rally_unforced1_2`: il giocatore 2 vince per un errore non forzato del giocatore 1.

- `C` = `rally_unforced2_1`: il giocatore 1 vince per un errore non forzato del giocatore 2.

Attualmente non ho aggiunto considerazioni sul primo e secondo servizio, e su posizionamenti della palla e tipologia di colpi giocati.

Ho considerato le challenges Hawk-Eye perse come errori forzati.
Errori forzati e winners sono stati considerati come la stessa cosa nei servizi, si potrebbe fare la stessa considerazione anche nei rally

In [31]:
for index, row in dataset.iterrows():
    winner = row["PtWinner"]

    #On first serve points
    if pd.isna(row['2nd']):
        numeri = [c for c in row['1st'] if c.isdigit()]
        ultimo = row['1st'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['1st'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'

    #On second serve points
    else:
        numeri = [c for c in row['2nd'] if c.isdigit()]
        ultimo = row['2nd'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['2nd'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'


dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,PointType
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,1,0,0,0,0.0,0-0,1,True,1,4n,4b1b1b3n@,NaN,1,B
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,2,0,0,0,0.0,15-0,1,True,1,5n,5f28f2b1b1b2f3f3d@,NaN,1,B
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,3,0,0,0,0.0,30-0,1,True,1,4C,NaN,NaN,1,3
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,4,0,0,0,0.0,40-0,1,True,1,4r2d#,NaN,NaN,1,9
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,5,0,0,1,0.0,0-0,2,True,2,5d,c4b1b1d!@,NaN,1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329842,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,119,0,1,3,5.0,AD-40,19,True,1,4d,4b3n#,NaN,1,9
329843,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,120,0,1,4,5.0,0-0,20,True,2,5d,6f1w#,NaN,2,A
329844,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,121,0,1,4,5.0,15-0,20,True,2,5s38b2f1f1f2b2b2f1f3b3b+1n@,NaN,NaN,2,C
329845,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,122,0,1,4,5.0,30-0,20,True,2,4f18f1f3b3s3b3s3f+1l1o3*,NaN,NaN,2,8


In [32]:
simboli_finali = Counter(dataset['PointType'])

for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'C': 52122
'B': 50625
'9': 49170
'A': 47474
'7': 37986
'8': 37134
'3': 21537
'4': 19926
'6': 6222
'5': 5894
'1': 882
'2': 875


Usare le informazioni precedenti per creare il vettore della progressione dei punti per ogni game accompagnato con un vettore avente la progressione dei game(questo servirà per predire la vittoria del set).

Inoltre servirà aggiungere 3 etichette, vincitore game, vincitore set e vincitore partita che serviranno successivamente.



COMPLETARE LOGICA DELLA PROGRESSIONE DEL SET, QUINDI DI COME SONO STATI VINTI I GAME. POTREBBE SERVIRE PER PREDIRE LA VITTORIA DEL SET

INFINE AGGIUNGERE LA LOGICA SULLE TIPOLOGIE DI PUNTI DEFINITE PRECEDENTEMENTE quindi con PoweredGmProgression

In [33]:
new_data = []
current_id = None
current_set = None
current_game = None
last_point = None
new_row = None

for index, row in dataset.iterrows():
    match_id = row['match_id']
    set_score = row['Set1'] + row['Set2']
    game_score = row['Gm1'] + row['Gm2']
    point_winner = row['PtWinner']
    point_type = row['PointType']

    # If New match
    if match_id != current_id:
        # Save previous Match labels
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point
              if r['MatchWinner'] == 0:
                  r['MatchWinner'] = last_point


        current_id = match_id
        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'OpGmProgression': [point_type],
            "Svr": row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If New set of the same Match
    elif set_score != current_set:
        # Save previous Set labels
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point



        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'OpGmProgression': [point_type],
            'Svr': row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If New game of the same Set
    elif game_score != current_game:
        # Save previous Game label
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)


        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'OpGmProgression': [point_type],
            'Svr': row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If same Game, Set and Match of the previous consideration
    else:
        # Save Game progression sequences
        if new_row:
            new_row['GmProgression'].append(point_winner)
            new_row['OpGmProgression'].append(point_type)

    last_point = point_winner

# Last dataset's game memorization
if new_row and last_point:
    new_row['GmWinner'] = last_point
    new_data.append(new_row)
    for r in new_data:
      if r['SetWinner'] == 0:
          r['SetWinner'] = last_point
      if r['MatchWinner'] == 0:
          r['MatchWinner'] = last_point




new_dataset = pd.DataFrame(new_data)
new_dataset

,match_id,GmProgression,OpGmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 1, 1]","[B, B, 3, 9]",1,0,0,0,0,1,1,1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 2, 2, 1, 2, 2, 2]","[B, B, 4, A, 9, 8, A, A]",2,0,0,1,0,2,1,1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[2, 1, 1, 1, 1]","[8, 7, B, 7, 3]",1,0,0,1,1,1,1,1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 2, 2, 1, 1, 2, 2, 2]","[B, A, 8, 7, B, 4, 8, 8]",2,0,0,2,1,2,1,1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 1, 1]","[9, 3, 3, 3]",1,0,0,2,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
53331,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[2, 2, 1, 1, 2, 2]","[A, A, B, 9, A, 8]",2,0,1,2,3,2,2,2
53332,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[1, 1, 1, 2, 2, 1]","[9, 9, 9, C, A, 9]",1,0,1,2,4,1,2,2
53333,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[2, 1, 2, 1, 2, 2]","[A, 5, C, 7, A, A]",2,0,1,3,4,2,2,2
53334,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[1, 1, 2, 2, 2, 1, 1, 1]","[B, 9, C, A, 8, 9, 3, 9]",1,0,1,3,5,1,2,2


In [34]:
simboli_finali = Counter(dataset['GmWinner'])

KeyError: 'GmWinner'